## Fazendo import das dependências

In [ ]:
from BHA_functions import asyncSimulations_Linux
from BHA_functions import DataCollector
from BHA_functions import GraphicGenerator
from datetime import datetime

## Definindo as constantes das simulações

In [ ]:
PATH = 'Simulations_Data/topology_simulations'
CORES = 12
RUNS_PER_POINT = 1

TARGET = (True, False)

NETWORK_PROB = 0.8
BHA_PROB = 0.4

TOPOLOGIES = ('Grade', 'Ba', 'Er')
BHA_NODES_PROP = (0.1, 0.3, 0.5)

PROB_EDGE_CREATION = 0.4
EDGES_TO_ATTACH = 3

NUMBER_OF_NODES = tuple([i for i in range(12, 108, 12)])
GRADE_NODES = {
    12:(3, 4),
    24:(4, 6),
    36:(6, 6),
    48:(6, 8),
    60:(6, 10),
    72:(8, 9),
    84:(7, 12),
    96:(8, 12)
}

GRAPHIC_POINTS = len(NUMBER_OF_NODES)

## Rodando a simulação sem Black Hole em todas as topologias

In [ ]:
# Default params
simulations_params = {
            'runs':RUNS_PER_POINT,
            'topology':TOPOLOGIES[0],
            'number_nodes':20,
            'rows':4,
            'columns':3,
            'prob_edge_creation':PROB_EDGE_CREATION,
            'edges_to_attach':EDGES_TO_ATTACH,
            'entanglements_replanished':10,
            'requests':100,
            'attempts_per_request':2,
            'network_prob':NETWORK_PROB,
            'num_black_holes':0,
            'black_hole_prob':BHA_PROB,
            'black_hole_target':TARGET
        }

for topology in TOPOLOGIES:
    simulations_params['topology'] = topology

    for number_of_nodes in NUMBER_OF_NODES:
        simulations_params['number_nodes'] = number_of_nodes

        if topology == 'Grade':
            simulations_params['rows'] = GRADE_NODES[number_of_nodes][0]
            simulations_params['columns'] = GRADE_NODES[number_of_nodes][1]

        # Runing no BHA simulation
        default_simulation = asyncSimulations_Linux(cores=CORES, **simulations_params)
        default_simulation.save(file_name=f'{PATH}/Default_Network/{topology}')

## Realizando todas as simulações com Black Holes

In [ ]:
for topology in TOPOLOGIES:
    simulations_params['topology'] = topology
    
    for target in TARGET:
        simulations_params['target'] = target
        
        for bha_prop in BHA_NODES_PROP:

            for point, number_of_nodes in enumerate(NUMBER_OF_NODES):
                start = datetime.now()
                
                if topology == 'Grade':
                    simulations_params['rows'] = GRADE_NODES[number_of_nodes][0]
                    simulations_params['columns'] = GRADE_NODES[number_of_nodes][1]

                # Updating umber of nodes of simulation
                simulations_params['num_black_holes'] = int(number_of_nodes * bha_prop)

                # Running simulation
                simulations_dc = asyncSimulations_Linux(cores=CORES, **simulations_params)

                # Saving CSV file
                simulations_dc.save(file_name=f'{PATH}/BHA_Network/{topology}/Target-{target}/{int(bha_prop*10)}prop/point{point}')

                # BenchMark
                print(f"As {simulations_params['runs']} simulações da topologia {topology} finalizaram no tempo de: {datetime.now()-start}")
                print('-='*50)

## Coletando os dados diretamente dos CSVs de cada simulação

In [ ]:
default_networks = dict()
for topology in TOPOLOGIES:
    default_network = DataCollector()
    default_network.get_DataFrame_csv(f'{PATH}/Default_Network/{topology}.csv')
    default_networks[topology] = default_network

datacollectors_list = []
for topology in TOPOLOGIES:
    for target in TARGET:
        for bha_prop in BHA_NODES_PROP:
            temp_datacollectors = []
            for point in range(0, GRAPHIC_POINTS):
                temp_dc = DataCollector()
                temp_dc.get_DataFrame_csv(f'{PATH}/BHA_Network/{topology}/Target-{target}/{bha_prop}nodes/point{point}.csv')
                temp_datacollectors.append(temp_dc)
            datacollectors_list.append(tuple(temp_datacollectors))

## Criando uma função genérica para plotar os gráficos

In [ ]:
def plotGraphic(
        dataCollectors_list: list[tuple[DataCollector]],
        default_diff: DataCollector, 
        intensity_variation: float,
        bha_nodes: tuple,
        y_column_name: str, 
        title: str, 
        x_label: str, 
        y_label: str) -> None:
    """
    Will show the simulations graph

    Args:
        dataCollectors_list (required): List with tuples of DataCollectors
        default_diff (required): Pass the network without black holes to compare, if None don't affect the result
        intensity_variation (required): Variation of intensity per point
        bha_nodes (required): Tuple with number of nodes
        y_column_name (required): Name of column data
        title (required): Title of graph
        x_label (required): X axis label
        y_label (required): Y axis label
    """
    
    graphicGen = GraphicGenerator()

    for pos, datacollectors in enumerate(dataCollectors_list):
        graphicGen.add_on_plot(
            plot_label=f'{bha_nodes[pos]} nós', 
            x_column=(0.1, intensity_variation), 
            y_column_name=y_column_name, 
            y_standard_deviation=False,
            dc=datacollectors,
            default_diff=default_diff)

    graphicGen.show_plot(
        title=title,
        x_label=x_label, 
        y_label=y_label)

## Gerando o gráfico da Topologia de Grade

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES_PROP,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Grade)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
    )

## Gerando o Gráfico da Topologia Barabasi-Albert

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES_PROP,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Barabasi-Albert)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
)

## Gerando o Gráfico da Topologia Erdos Renyi

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES_PROP,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Erdos Renyi)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
)